In [1]:
# Image size (height x width)
ih = 64
iw = 64

# Grayscale or RGB
ch = 'rgb'

# Batch size
batch_size = 1024

# Number of epochs
epoch_t = 35

# number of folds
n_folds = 10

In [2]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from sklearn.model_selection import KFold
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import copy

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
# Define transform to resize images to 224x224 and normalize pixel values
transform = transforms.Compose([
    transforms.Resize((ih,iw),transforms.InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [5]:
# Data directory
root_path = "/home/max/Projects/MachineLearning/RansomwareDetection/data/"

# Define dataset by specifying the path to the root directory
dataset = datasets.ImageFolder(root=root_path, transform=transform)
num_classes = len(dataset.classes)


## DenseNet Model

In [6]:
base_model = models.densenet169(weights="IMAGENET1K_V1")

In [7]:
for param in base_model.parameters():
    param.requires_grad = False

# Add the classification layers for this dataset
num_ftrs = base_model.classifier.in_features
head_model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(6656,1000),
    nn.ReLU(inplace=True),
    nn.Linear(1000,num_classes),
    nn.Softmax(dim=1)
)

# Combine the base and head models
model = nn.Sequential(
    base_model.features,
    head_model
)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))

# Freeze all but last eight layers
for param in model.parameters():
    param.requires_grad = False

# Freeze all but last eight layers
params_to_update = list(model.parameters())[-8:]
for param in params_to_update:
    param.requires_grad = True


In [8]:
# Replace the last fully-connected layer with a new one that has the same number of outputs as your dataset

# model.classifier = nn.Linear(1000, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.to(device)

Sequential(
  (0): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): R

In [10]:
import time
# Set up cross-validation
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

scaler = torch.cuda.amp.GradScaler()

for fold, (train_indices, test_indices) in enumerate(kf.split(dataset)):
    # Define data loaders for training and testing subsets
    train_subset = torch.utils.data.Subset(dataset, train_indices)
    test_subset = torch.utils.data.Subset(dataset, test_indices)
    train_loader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=True,num_workers=8,pin_memory=True)
    test_loader = torch.utils.data.DataLoader(test_subset, batch_size=batch_size, shuffle=True,num_workers=8, pin_memory=True)
    
    # Reinitialize the model weights

    # Save best model performance
    best_model_weights = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(epoch_t):
        start = time.time()
        print(f'Epoch {epoch}/{epoch_t - 1}')
        print('-' * 10)
        model.train()
        running_loss=0.0

        # Each epoch has a training and validation phase
        for inputs, labels in train_loader:

            inputs, labels = inputs.to(device), labels.to(device)
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Casts operations to mixed precision
            with torch.cuda.amp.autocast():
                # Forward pass
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            

            # Backward pass and optimize
            scaler.scale(loss).backward()

            scaler.step(optimizer)

            scaler.update()

            # Accumulate loss
            running_loss += loss.item()
        
        # Evaluate the model on the validation set
        model.eval()  # Set model to evaluation mode
        accuracy = 0.0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predictions = torch.max(outputs, 1)
                accuracy += (predictions == labels).sum().item() / len(test_indices)

        # Print performance metrics
        epoch_loss = running_loss / len(train_indices)
        print(f'Fold {fold + 1}, Epoch {epoch + 1}: loss={epoch_loss:.4f}, accuracy={accuracy:.4f}, time: {time.time()-start}')



Epoch 0/34
----------
Fold 1, Epoch 1: loss=0.0017, accuracy=0.8535, time: 5.768320083618164
Epoch 1/34
----------
Fold 1, Epoch 2: loss=0.0017, accuracy=0.8561, time: 6.159991025924683
Epoch 2/34
----------
Fold 1, Epoch 3: loss=0.0017, accuracy=0.8580, time: 5.2455832958221436
Epoch 3/34
----------
Fold 1, Epoch 4: loss=0.0017, accuracy=0.8588, time: 5.829010009765625
Epoch 4/34
----------
Fold 1, Epoch 5: loss=0.0017, accuracy=0.8580, time: 7.245812177658081
Epoch 5/34
----------


KeyboardInterrupt: 